## Heat tranfer in pipes

This notebooks helps to define better initial estimates to permit an easy convergence of the DAETools.

#### Specifications

In [121]:
p1 = 9.95*1e5 # Pa - upstrem pressure
v = 1.0 # m/s - fluid velocity
T = 300 # K - fluid temperature
pi = 3.14

#### Parameters

In [158]:
Di = 1.0 * 4.026*0.0254 # m
Do = 1.1 * 4.026*0.0254 # m
ep = 0.0018*0.0254 # m
kwall = 51.9 # (K ** (-1))*(J ** (1))*(s ** (-1))*(m ** (-1))
Text = 273 + 60. #K
hext = 12566.787163427189 # (K ** (-1))*(J ** (1))*(s ** (-1))*(m ** (-2))
L = 100 # m
g = 9.81
rhov = 1/7.650
hvap = 2358400
Tsat = Text
Ntub = 1

# Properties
from water_properties import density, viscosity, conductivity, heat_capacity
rho = density(T, p1, simplified=True)
mu = viscosity(T, p1, simplified=True)
kappa = conductivity(T, p1, simplified=True)
cp = heat_capacity(T, p1, simplified=True)
prandtl = cp * mu / kappa

# Reynolds
D = Di
Re = D * v * rho / mu

print("rho = {0}".format(rho))
print("mu = {0}".format(mu))
print("kappa = {0}".format(kappa))
print("cp = {0}".format(cp))

rho = 996.937908321761
mu = 0.0008582410285531535
kappa = 0.6099612012445662
cp = 4178.757816453117


#### Friction factor calculation

In [123]:
from scipy.optimize import root
import numpy as np

def darcy(ifD, ep, D, Re):
    res = ifD + 2. * np.log10(ep / 3.72 / D + (2.51 / Re) * ifD )
    return res

def calculate_darcy(fD0, ep, D, Re):
    ifD0 = fD0 ** -0.5
    sol = root(darcy, ifD0, args=(ep, D, Re),)
    ifD = sol.x[0]
    fD = ifD ** -2
    res = darcy(ifD, ep, D, Re)
    return (fD, res)

fD0 = 0.018    
fD, res = calculate_darcy(fD0, ep, D, Re)
print("The Reynolds number is: {0}".format(Re))
print("The rougthness is {0} m".format(ep))
print("The flow diameter is {0} m".format(D))
print("The calculated friction factor is: {0}".format(fD))
print("The residue is {0}".format(res))



The Reynolds number is: 118786.29183227483
The rougthness is 4.5719999999999996e-05 m
The flow diameter is 0.10226039999999999 m
The calculated friction factor is: 0.019647235607534896
The residue is 8.881784197001252e-16


#### Calculate pressure

In [124]:
hL = 0.5 * fD * v ** 2 /  (D * g )
DeltaP = g * rho * hL
p2 = p1 - DeltaP * L
A = 0.25 * 3.14 * D **2 
q = rho * v * A
print("The pressure loss is equal to {0} Pa/m".format(DeltaP))
print("The upstream pressure is equal to {0} Pa".format(p1))
print("The downstream pressure is equal to {0} Pa".format(p2))
print("The mass flowrate is equal to {0} kg/s".format(q))

The pressure loss is equal to 95.77057184834338 Pa/m
The upstream pressure is equal to 994999.9999999999 Pa
The downstream pressure is equal to 985422.9428151655 Pa
The mass flowrate is equal to 8.18375730036397 kg/s


## Case A - Fixed External Temperature

#### External convection coefficient

In [125]:
Resext = 0.0
print("The calculated internal resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Resext))

The calculated internal resistance is 0.0 (K ** (1))*(W ** (-1))*(m ** (1))


#### Internal convection coefficient

In [126]:
# Calculates the Nussel dimensionless number using Petukhov correlation modified by Gnielinski. See Incropera 4th Edition [8.63]
nusselt = (fD / 8.) * (Re - 1000.) * prandtl / ( 1. + 12.7 * (fD / 8.) ** 0.5 * (prandtl ** 2 / 3) - 1.)
hint = nusselt * kappa / D
Resint = 1 / (2 * pi * D * hint)
print("The calculated internal convection coeffient is {0} (K ** (-1))*(W ** (1))*(m ** (-2))".format(hint))
print("The calculated internal resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Resint))

The calculated internal convection coeffient is 1398.8091902015963 (K ** (-1))*(W ** (1))*(m ** (-2))
The calculated internal resistance is 0.0011132030616867582 (K ** (1))*(W ** (-1))*(m ** (1))


#### Wall conduction

In [127]:
Reswall = np.log(Do / Di) / (2 * pi * kwall)
print("The calculated wall resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Reswall))

The calculated wall resistance is 0.00029242351105238253 (K ** (1))*(W ** (-1))*(m ** (1))


#### Total Thermal resistance 

In [128]:
Restotal = Resext + Resint + Reswall
print("The total heat transfer resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Restotal))

The total heat transfer resistance is 0.0014056265727391407 (K ** (1))*(W ** (-1))*(m ** (1))


#### Temperature profiles

In [129]:
Qout = (T - Text) / Restotal
T = T
Ti = T - Qout*Resint
To = Text + Qout*Resext
print("The total heat transfered is {0} (K ** (-1))*(W ** (1))*(m ** (-1))".format(Qout))
print("The temperature T is {0} K".format(T))
print("The temperature Ti is {0} K".format(Ti))
print("The temperature To is {0} K".format(To))
print("The temperature Text is {0} K".format(Text))

The total heat transfered is -23477.07466549454 (K ** (-1))*(W ** (1))*(m ** (-1))
The temperature T is 300 K
The temperature Ti is 326.13475139707714 K
The temperature To is 333.0 K
The temperature Text is 333.0 K


## Case B - Fixed External Convection

#### External convection coefficient

In [153]:
hext = 6872.579071624325
Resext = 1 / (2 * pi * D * hext)
print("The calculated external convection coeffient is {0} (K ** (-1))*(W ** (1))*(m ** (-2))".format(hext))
print("The calculated external resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Resext))

The calculated external convection coeffient is 6872.579071624325 (K ** (-1))*(W ** (1))*(m ** (-2))
The calculated external resistance is 0.00022657559222231832 (K ** (1))*(W ** (-1))*(m ** (1))


#### Internal convection coefficient

In [154]:
# Calculates the Nussel dimensionless number using Petukhov correlation modified by Gnielinski. See Incropera 4th Edition [8.63]
nusselt = (fD / 8.) * (Re - 1000.) * prandtl / ( 1. + 12.7 * (fD / 8.) ** 0.5 * (prandtl ** 2 / 3) - 1.)
hint = nusselt * kappa / D
Resint = 1 / (2 * pi * D * hint)
print("The calculated internal convection coeffient is {0} (K ** (-1))*(W ** (1))*(m ** (-2))".format(hint))
print("The calculated internal resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Resint))

The calculated internal convection coeffient is 1398.8091902015963 (K ** (-1))*(W ** (1))*(m ** (-2))
The calculated internal resistance is 0.0011132030616867582 (K ** (1))*(W ** (-1))*(m ** (1))


#### Wall conduction

In [155]:
Reswall = np.log(Do / Di) / (2 * pi * kwall)
print("The calculated wall resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Reswall))

The calculated wall resistance is 0.00029242351105238253 (K ** (1))*(W ** (-1))*(m ** (1))


#### Total Thermal resistance 

In [156]:
Restotal = Resext + Resint + Reswall
print("The total heat transfer resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Restotal))

The total heat transfer resistance is 0.0016322021649614591 (K ** (1))*(W ** (-1))*(m ** (1))


#### Temperature profiles

In [157]:
Qout = (T - Text) / Restotal
T = T
Ti = T - Qout*Resint
To = Text + Qout*Resext
print("The total heat transfered is {0} (K ** (-1))*(W ** (1))*(m ** (-1))".format(Qout))
print("The temperature T is {0} K".format(T))
print("The temperature Ti is {0} K".format(Ti))
print("The temperature To is {0} K".format(To))
print("The temperature Text is {0} K".format(Text))

The total heat transfered is -20218.08370826369 (K ** (-1))*(W ** (1))*(m ** (-1))
The temperature T is 300 K
The temperature Ti is 322.5068326854783 K
The temperature To is 328.41907571019976 K
The temperature Text is 333.0 K


## Case C - External film condensation

#### External convection coefficient

In [159]:
rho_o = density(To, p1, simplified=True)
kappa_o = conductivity(To, p1, simplified=True)
mu_o = viscosity(To, p1, simplified=True)

num = (g * rho_o * (rho_o - rhov) * kappa_o ** 3. * hvap)
den = mu_o * (Tsat - To) * Do

hd1 = 0.729 * (num / den) ** 0.25

# Correction factor for multiple tubes
f = Ntub ** 0.75 - (Ntub - 1) ** 0.75

hext = f * hd1
Resext = 1 / (2 * pi * D * hext)
print(num, den, hd1)
print("The calculated external convection coeffient is {0} (K ** (-1))*(W ** (1))*(m ** (-2))".format(hext))
print("The calculated external resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Resext))

6086719006114.879 0.000255143755439424 9059.977219137703
The calculated external convection coeffient is 9059.977219137703 (K ** (-1))*(W ** (1))*(m ** (-2))
The calculated external resistance is 0.00017187225040243495 (K ** (1))*(W ** (-1))*(m ** (1))


#### Internal convection coefficient

In [149]:
# Calculates the Nussel dimensionless number using Petukhov correlation modified by Gnielinski. See Incropera 4th Edition [8.63]
nusselt = (fD / 8.) * (Re - 1000.) * prandtl / ( 1. + 12.7 * (fD / 8.) ** 0.5 * (prandtl ** 2 / 3) - 1.)
hint = nusselt * kappa / D
Resint = 1 / (2 * pi * D * hint)
print("The calculated internal convection coeffient is {0} (K ** (-1))*(W ** (1))*(m ** (-2))".format(hint))
print("The calculated internal resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Resint))

The calculated internal convection coeffient is 1398.8091902015963 (K ** (-1))*(W ** (1))*(m ** (-2))
The calculated internal resistance is 0.0011132030616867582 (K ** (1))*(W ** (-1))*(m ** (1))


#### Wall conduction

In [150]:
Reswall = np.log(Do / Di) / (2 * pi * kwall)
print("The calculated wall resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Reswall))

The calculated wall resistance is 0.00029242351105238253 (K ** (1))*(W ** (-1))*(m ** (1))


#### Total Thermal resistance 

In [151]:
Restotal = Resext + Resint + Reswall
print("The total heat transfer resistance is {0} (K ** (1))*(W ** (-1))*(m ** (1))".format(Restotal))

The total heat transfer resistance is 0.0016322021649614591 (K ** (1))*(W ** (-1))*(m ** (1))


#### Temperature profiles

In [152]:
Qout = (T - Text) / Restotal
T = T
Ti = T - Qout*Resint
To = Text + Qout*Resext
print("The total heat transfered is {0} (K ** (-1))*(W ** (1))*(m ** (-1))".format(Qout))
print("The temperature T is {0} K".format(T))
print("The temperature Ti is {0} K".format(Ti))
print("The temperature To is {0} K".format(To))
print("The temperature Text is {0} K".format(Text))

The total heat transfered is -20218.08370826369 (K ** (-1))*(W ** (1))*(m ** (-1))
The temperature T is 300 K
The temperature Ti is 322.5068326854783 K
The temperature To is 328.41907571019976 K
The temperature Text is 333.0 K
